### 한국가스공사 수소충전소 현황
* 수소유통센터에서 조사된 수소충전소 위치 및 가격, 영업시간 등 수소충전소 운영현황으로 수소 차 이용자들의 편의 증대 및 수소 충전소 보급 현황 조회 가능
* 기타 유의사항 : 이용가능 요일은 운영일은 1 휴무일은 0으로 (월/화/수/목/금/토/일/공휴일) 8자리로 표시됩니다. (예시 : 평일운영, 주말 휴무, 공휴일 휴무인 경우 11111000) 추가적으로, 충전소에서 귀 기관으로 제공하지 않는 정보(예 : 전화번호)는 공란으로 되어있으므로 참고 바랍니다.
* 공공데이터포털 출처 자료이나 현 시점에서는 제공되지 않아 대체 데이터 링크를 제공합니다. 대체 데이터보다 다양한 변수를 제공하기 때문에 해당 수소충전소현황 데이터를 사용합니다. https://www.data.go.kr/data/15090186/fileData.do

In [ ]:
# 군집분석, folium 지도시각화를 위한 라이브러리 설치가 필요합니다.
# !pip install -Uq koreanize-matplotlib scikit-learn folium

In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

### 데이터로드

In [ ]:
df = pd.read_csv("hydrogen-station.csv", encoding="cp949")
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

### 기초 기술 통계

In [ ]:
df.describe().style.format("{:.0f}")

In [ ]:
df.describe(exclude="number")

In [ ]:
# df = raw.melt(id_vars=['순번', '충전소_관리번호', '충전소_명', '전화번호', '도로명주소', '지번주소', '충전소_유형코드',
#        '충전소_유형명', '충전소유형비고', '충전가능차량코드', '충전가능차량', '판매가격', '이용가능요일', '경도', '위도'],
#               var_name="요일", value_name="시간")
# df.shape

In [ ]:
df

### 정규표현식을 사용한 전화번호 마스크처리

In [ ]:
# 정규표현식을 사용하여 가운데 번호를 마스크 처리하는 함수
def mask_middle_number(phone_number):
    if not pd.isna(phone_number):
        return re.sub(r'(\d{2,4}-)\d{2,4}(-\d{4})', r'\1****\2', phone_number)

# 전화번호 컬럼에 함수 적용
df['전화번호마스크'] = df['전화번호'].apply(mask_middle_number)
df[['전화번호', '전화번호마스크']]

### 시도명 파생변수 만들기

In [ ]:
# 시도명 추출 함수
def extract_sido(address):
    if pd.isna(address):
        return 'N/A'
    # 공백을 기준으로 분할하여 첫 번째 요소를 시도명으로 추출
    return address.split()[0]

# 시도명 추출
df['시도명'] = df['도로명주소'].apply(extract_sido)

In [ ]:
pd.crosstab(df["시도명"], df["충전소_유형명"])

### 충전소유형비고, 충전가능차량에 대한 교차표 생성

In [ ]:
pd.crosstab(df["충전소유형비고"], df['충전가능차량'])

In [ ]:
# 충전소유형비고 열의 값을 쉼표(,)로 분리하여 새로운 행으로 확장
df_expanded = df['충전소유형비고'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).str.strip()
df_expanded = df_expanded.reset_index().rename(columns={0: '충전소유형비고'})
df_expanded

In [ ]:
# 충전가능차량 열을 원래의 인덱스를 기준으로 병합
df_expanded = df_expanded.join(df[['시도명', '충전가능차량']], on='index')
df_expanded

In [ ]:
# 교차표 생성
pd.crosstab(df_expanded["충전소유형비고"], df_expanded['충전가능차량'])

In [ ]:
pd.crosstab(df_expanded['시도명'], df_expanded["충전소유형비고"])

# Prompt : 
```
같은 시도인데도 '경기', '경기도' 처럼 다른 명칭으로 되어 있는 시도가 많습니다. 일관되게 작성하여 교차표를 작성했을 때 다른 시도로 집계되지 않도록 코드를 작성해 주세요.

array(['인천', '서울', '경북', '충청북도', '전북', '충남', '경남', '충북', '울산', 'N/A',
       '대구', '대전', '경기', '경상남도', '전남', '광주', '부산', '강원특별자치도', '경기도',
       '부산광역시', '인천광역시', '전라북도', '서울특별시', '충청남도', '대전광역시', '광주광역시',
       '울산광역시', '대구광역시', '세종특별자치시', '전라남도'], dtype=object)
```

In [ ]:
df["시도명"].unique()

In [ ]:
# 시도명 표준화 사전
standardize_sido = {
    '서울특별시': '서울',
    '부산광역시': '부산',
    '대구광역시': '대구',
    '인천광역시': '인천',
    '광주광역시': '광주',
    '대전광역시': '대전',
    '울산광역시': '울산',
    '세종특별자치시': '세종',
    '경기': '경기도',
    '경기도': '경기도',
    '충청북도': '충북',
    '충북': '충북',
    '충청남도': '충남',
    '충남': '충남',
    '전라북도': '전북',
    '전북': '전북',
    '전라남도': '전남',
    '전남': '전남',
    '경상북도': '경북',
    '경북': '경북',
    '경상남도': '경남',
    '경남': '경남',
    '강원특별자치도': '강원'
}

# 시도명을 표준화
df_expanded['시도'] = df_expanded['시도명'].map(standardize_sido)

In [ ]:
pd.crosstab(df_expanded['시도'], df_expanded["충전소유형비고"])

### 충전소 위치, 차량 종류에 따른 판매가격 분석

In [ ]:
# 시도명, 차량 종류별 평균 판매가격 계산
avg_price_by_location_type_vehicle = df.groupby(['시도', '충전가능차량'])['판매가격'].mean().reset_index()
avg_price_by_location_type_vehicle.nlargest(10, "판매가격")

In [ ]:
avg_price_by_location_type_vehicle.nsmallest(10, "판매가격")

### 군집 분석
* https://scikit-learn.org/stable/modules/clustering.html#k-means
* https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

In [ ]:
# 라이브러리 설치가 필요합니다.
# !pip install -Uq scikit-learn

In [ ]:
from sklearn.cluster import KMeans
# K-means 클러스터링
kmeans = KMeans(n_clusters=7, n_init="auto", random_state=42)
df['cluster'] = kmeans.fit_predict(df[['경도', '위도']])
df

In [ ]:
# 군집별 평균 판매가격 계산
cluster_mean_price = df.groupby('cluster')['판매가격'].mean()
cluster_count = df.groupby('cluster')['판매가격'].count()

# 군집별 주요 시도 찾기
cluster_major_region = df.groupby('cluster')['시도'].apply(lambda x: ", ".join(x.value_counts().index[:5].tolist()))

# 결과를 데이터프레임으로 만들기
result_df = pd.DataFrame({
    '주요_시도': cluster_major_region,
    '평균_판매가격': cluster_mean_price,
    '충전소수' : cluster_count
})

# 결과 출력
result_df

In [ ]:
df[["위도", "경도"]].describe()

In [ ]:
sns.scatterplot(data=df, x="경도", y="위도", hue="cluster")

### folium 을 통한 지도 시각화

[Getting started — Folium documentation](https://python-visualization.github.io/folium/latest/getting_started.html)

In [ ]:
# !pip install -Uq scikit-learn

In [ ]:
folium_center = df[["위도", "경도"]].mean().tolist()
folium_center

In [ ]:
import folium
# 지도 생성
fm = folium.Map(location=folium_center, zoom_start=7)

# 클러스터별 색상 정의
cluster_colors = ['red', 'blue', 'green', 'purple', 'orange', 'pink', 'darkblue']

# 충전소 위치에 마커 추가
for idx, row in df.iterrows():
    tooltip_desc = f"{row['충전소_명']} - {row['도로명주소']}"
    color = cluster_colors[row["cluster"]]
    folium.Marker(
        location=[row['위도'], row['경도']],
        tooltip=tooltip_desc,
        icon=folium.Icon(color=color, icon='charging-station', prefix='fa')
    ).add_to(fm)

# 지도 저장
fm.save('charging_stations_map.html')

# 지도 표시 (주피터 노트북에서만 사용)
fm